# TF-IDF with Scikit Learn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
import nltk
import glob
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings('ignore')

In [2]:
# load all the dataframes
country_df = pd.read_csv('data/Country-lyrics.csv', index_col=0)
hip_hop_df = pd.read_csv('data/Hip-Hop-Rnb-lyrics.csv', index_col=0)
pop_df = pd.read_csv('data/pop-lyrics.csv', index_col=0)
christian_df = pd.read_csv('data/Christian-lyrics.csv', index_col=0)
electro_df = pd.read_csv('data/Electro-lyrics.csv', index_col=0)
rock_df = pd.read_csv('data/Rock-lyrics.csv', index_col=0)

In [3]:
# empyty list for documents
documents = []

In [4]:
# choose a genre to analyze. for example 'country_df' for country music
genre = country_df

In [5]:
# move the words out of pandas dataframe into a list of documents
for key, values in genre.groupby(['artist', 'title', 'year']):
    documents.append(values['word'].astype('str').values)

documents = [' '.join(doc) for doc in documents]

In [6]:
# remove song title and 'lyrics' from start of each document
# remove 'embed' that is at the end of each document
# remove punctuation but keep apostrophes
documents = [re.sub("\d{1,4}Embed|Embed|^.*?(Lyrics)|[^\w\d'\s]+" , '', doc) for doc in documents]

In [7]:
def lemmatization(documents, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in documents:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [8]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    max_features=100,
    max_df=0.8,
    min_df=5,
    ngram_range=(1, 5),
    stop_words='english'
)

In [9]:
lemmatized_documents = lemmatization(documents)

vectors = vectorizer.fit_transform(lemmatized_documents)

feature_names = vectorizer.get_feature_names_out()

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

In [10]:
for description in denselist:
    i = 0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[i])
        i += 1
    all_keywords.append(keywords)

In [11]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(all_keywords)

texts = all_keywords

corpus = [id2word.doc2bow(text) for text in texts]

In [12]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x             y  topics  cluster       Freq
topic                                                    
5     -0.077954  4.599918e-02       1        1  21.412755
8      0.075734  6.192737e-02       2        1  18.660190
7     -0.033416  1.108734e-01       3        1  15.429164
3     -0.153713 -1.131394e-01       4        1  10.946986
0     -0.169623  2.406286e-01       5        1   9.567430
1      0.117332  2.569125e-01       6        1   8.156371
6      0.058194 -1.265708e-01       7        1   7.862097
4     -0.305794 -9.987308e-03       8        1   5.141536
9      0.483353  6.488670e-07       9        1   1.501066
2      0.005887 -4.666441e-01      10        1   1.322403, topic_info=     Term        Freq       Total Category  logprob  loglift
31    old  243.000000  243.000000  Default  30.0000  30.0000
5    hand  216.000000  216.000000  Default  29.0000  29.0000
87   line  143.000000  143.000000  Default  28.0000  28.0000
96    run  140.000000  140.000000  Default  27.0000  27.0000
29    man  174.000000  174.000000  Default  26.0000  26.0000
..    ...         ...         ...      ...      ...      ...
10  night    0.090997  397.434790  Topic10  -8.0926  -4.0562
22   head    0.090994  126.037911  Topic10  -8.0926  -2.9078
52    say    0.090989  458.096983  Topic10  -8.0927  -4.1984
40   time    0.090984  471.654687  Topic10  -8.0928  -4.2276
9     let    0.090984  360.581612  Topic10  -8.0928  -3.9591

[416 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
95        1  0.463751   away
95        3  0.077292   away
95        4  0.237396   away
95        6  0.176667   away
95        8  0.044167   away
...     ...       ...    ...
65        5  0.079522  world
65        7  0.223657  world
65        8  0.228627  world
76        1  0.457487  wrong
76        6  0.527105  wrong

[463 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 8, 4, 1, 2, 7, 5, 10, 3])